In [ ]:
#import required libraries
from azure.ml import MLClient
from azure.ml.entities import CommandJob, JobInput, SweepJob, Choice, Uniform, SweepJobLimits, TruncationSelectionPolicy, Objective, CommandComponent
from azure.identity import InteractiveBrowserCredential

In [ ]:
#Enter details of your AML workspace
subscription_id = '<SUBSCRIPTION_ID>'
resource_group = '<RESOURCE_GROUP>'
workspace = '<AML_WORKSPACE_NAME>'

In [ ]:
#get a handle to the workspace
ml_client = MLClient(InteractiveBrowserCredential(), subscription_id, resource_group, workspace)

## Run a command job to train a LightGBM model on the Iris dataset.

In [ ]:
#define the command job
command_job=CommandJob(
    code_local_path='./src',
    command = 'python main.py --iris-csv ${{inputs.iris_csv}} --learning-rate ${{inputs.learning_rate}} --boosting ${{inputs.boosting}}',
    environment='AzureML-lightgbm-3.2-ubuntu18.04-py37-cpu:11',
    inputs={'iris_csv':JobInput(file='https://azuremlexamples.blob.core.windows.net/datasets/iris.csv'),'learning_rate': 0.9, 'boosting': 'gbdt'},
    compute='cpu-cluster',
    display_name='lightgbm-iris-example',
    experiment_name='lightgbm-iris-example',
    description='Train a LightGBM model on the Iris dataset.'
)

#submit the command job
returned_job = ml_client.create_or_update(command_job)
#get a URL for the status of the job
returned_job.services["Studio"].endpoint


## Run a sweep on this command job
We will now sweep this command job and supply it with different sets of parameters from the search space

In [ ]:
#define the search space for this sweep. Parameters provided in search space will override parameters provided as inputs within the command job itself
# Each trial job will be provided with a different combination of hyperparameter values that the system samples from the search_space. 
search_space = {'learning_rate': Uniform(min_value=0.01, max_value=0.9), 'boosting': Choice(values=['gbdt', 'dart'])}

#define the limits for this sweep
limits = SweepJobLimits(max_total_trials=20, max_concurrent_trials=10, timeout=7200)

In [ ]:
#define the sweep job
sweep_job = SweepJob(
    trial=command_job,
    compute='cpu-cluster',
    sampling_algorithm='random',
    search_space=search_space,
    objective=Objective(goal='Minimize', primary_metric='test-multi_logloss'),
    limits=limits,
    display_name='lightgbm-iris-sweep-example',
    experiment_name='lightgbm-iris-sweep-example',
    description='Run a hyperparameter sweep job for LightGBM on Iris dataset.'
)

In [ ]:
#submit the sweep job
returned_sweep_job = ml_client.create_or_update(sweep_job)
#get a URL for the status of the job
returned_sweep_job.services["Studio"].endpoint

## Sweep using a component
We can use a reusable command component for the trial to run a sweep instead of a command job

In [ ]:
#define the trial component
from azure.ml.entities import Component
parent_dir = '.'
trial_component = Component.load(path=parent_dir + "/train.yml")

In [ ]:
# run sweep using this component
inputs = {'iris_csv': JobInput(file='wasbs://datasets@azuremlexamples.blob.core.windows.net/iris.csv',mode='rw_mount')}
cmd_sweep_job = SweepJob(
    trial=trial_component,
    compute='cpu-cluster',
    sampling_algorithm='random',
    inputs=inputs,
    search_space=search_space,
    objective=Objective(goal='Minimize', primary_metric='test-multi_logloss'),
    limits=limits,
    display_name='lightgbm-iris-sweep-component-example',
    experiment_name='lightgbm-iris-sweep-component-example',
    description='Run a hyperparameter sweep job using component for LightGBM on Iris dataset.'
)

In [ ]:
#submit the sweep job
returned_sweep_job_cmd = ml_client.create_or_update(cmd_sweep_job)
#get a URL for the status of the job
returned_sweep_job_cmd.services["Studio"].endpoint